high rez ver

In [1]:
import sounddevice as sd
import numpy as np
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# Pretrained model loading
tokenizer = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-korean")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-korean")

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# pyaudio params
CHUNK = 2048
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100

# speech recognition function
def recognize_speech_from_mic():
    p = pyaudio.PyAudio()
    frames = []
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")
    for i in range(0, int(RATE / CHUNK * 5)):
        data = stream.read(CHUNK)
        frames.append(np.frombuffer(data, dtype=np.int16))

    print("* done recording")

    stream.stop_stream()
    stream.close()
    p.terminate()


    frame = np.concatenate(frames).astype(np.float32)
    input_values = tokenizer(frame, return_tensors="pt", padding="longest", sampling_rate=RATE).input_values
    input_values = input_values.to(device)



    with torch.no_grad():
        logits = model(input_values).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.decode(predicted_ids[0])
    print("Transcription: ", transcription)



if __name__ == "__main__":
    recognize_speech_from_mic()


/Users/cafalena/opt/anaconda3/envs/torchenv/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:53: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(


OSError: facebook/wav2vec2-large-960h-korean is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.